In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Bidirectional, SimpleRNN
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
from nltk.stem import ISRIStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics import accuracy_score, classification_report

In [2]:
file_path = "train.xlsx"
df_train = pd.read_excel(file_path)
df_train

,review_description,rating
0,شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...,-1
1,خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...,1
2,تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...,-1
3,فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...,1
4,سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...,-1
...,...,...
32031,التطبيق اصبح سيء للغايه نقوم بطلب لا يتم وصول ...,-1
32032,y love you,1
32033,الباقه بتخلص وبشحن مرتين باقه اضافيه ١٠٠ جنيه,-1
32034,تطبيق فاشل وصلني الطلب ناقص ومش ينفع اعمل حاجة...,-1


In [3]:
arabic_stop_words = [
    "و", "في", "من", "على", "إلى", "لا", "أو", "هو", "هي", "يكون",
    "أنا", "أنت", "هو", "هي", "نحن", "أنتم", "هم",
    "عن", "مع", "كما", "مثل", "بين", "إذا", "حتى", "منذ",
    "و", "أو", "لكن", "إذا", "إن",
    "اليوم", "غداً", "الآن", "ثم", "بعد",
    "كان", "يكون", "أصبح", "صار", "ليس", "لم",
    "هذا", "هذه", "ذلك", "تلك", 
    "كل", "على", "فيه", "منه", "عنه", "له", "به", "إليه", "لها", "فيها",
    "بها", "منها", "عنها", "إليها", "الذي", "التي", "اللذين", "اللذان", "اللتان",
    "اللتين", "هؤلاء", "ذلك", "هذه", "هذا", "تلك", "تحت", "فوق", "معه", "لديه",
    "عليه", "عليها", "أي", "هل", "إذا", "ماذا", "هناك", "هنالك", "إلى",
    "يناير", "فبراير", "مارس", "إبريل", "مايو", "يونيو", "يوليو", "أغسطس", "سبتمبر", "أكتوبر", "نوفمبر", "ديسمبر",
    "الأحد", "الاثنين", "الثلاثاء", "الأربعاء", "الخميس", "الجمعة", "السبت"
]

In [4]:
import string


def text_cleaning(text, stemmer=ISRIStemmer()):
    # remove stop words and punctuation
    text = text.lower()
    tokenizer = RegexpTokenizer(r"\w+")
    words = tokenizer.tokenize(text)

    stop_words = set(string.punctuation).union(set(arabic_stop_words))

    filtered_list = [word for word in words if word.casefold() not in stop_words]

    # word stemming
    stem_words = [stemmer.stem(word) for word in filtered_list]

    return " ".join(map(str, stem_words))


df_train["new review_description"] = df_train["review_description"].apply(
    lambda text: text_cleaning(text)
)
df_train.head(20)

,review_description,rating,new review_description
0,شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...,-1,شرك زبل سوق رشم فيش حتي رقم شكو سوق يسب يمش مي...
1,خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...,1,خدم دفع طرق الك نت وقف عند اصبح فقط دفع نقد
2,تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...,-1,طبق غبي جري حذف عمل كود خصم لما خدم كتر مرة عم...
3,فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...,1,فعل طبق متز بس لو فى مكن يتح خدم طبق ان تطع غي...
4,سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...,-1,سيء جدا سعر رسم وصل تمت وقع ب صله
5,قعد عشرين سنة يدور على سائق بس اما عن توصيل ال...,0,قعد عشر سنة يدر سئق بس اما وصل شيء جيد جدا
6,احلئ تطبيق,1,حلئ طبق
7,رائع واو مدهش,1,رئع واو دهش
8,مکو بس البحرین وعمان وغیرهه بس العراق مکو یعنی...,-1,مکو بس حری وعم غیر بس عرق مکو یعنی نجم وحد علی...
9,تطبيق جميل يستاهل الخمس نجوم👍👍👍,1,طبق جمل تهل خمس نجم


In [5]:
X = df_train["new review_description"]
y = df_train["rating"].astype(int) + 1

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
total_words = len(tokenizer.word_index) + 1

In [7]:
sequences = tokenizer.texts_to_sequences(X)
max_len = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_len)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(
    padded_sequences, y, test_size=0.2, random_state=42, stratify=y
)
num_classes = 3

In [9]:
model = Sequential()
model.add(
    Embedding(
        input_dim=len(tokenizer.word_index) + 1,
        output_dim=300,
        input_length=X_train.shape[1],
    )
)
model.add(LSTM(32, activation="tanh", dropout=0.5))
model.add(Dense(num_classes, activation="softmax"))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

checkpoint = ModelCheckpoint(
    filepath="./models_hdf5/LSTM-emb.hdf5",
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,
)

In [10]:
model.fit(
    X_train,
    y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[checkpoint],
)

Epoch 1/5
970/970 [==============================] - 41s 36ms/step - loss: 0.5192 - accuracy: 0.8045 - val_loss: 0.4395 - val_accuracy: 0.8420
Epoch 2/5
970/970 [==============================] - 34s 35ms/step - loss: 0.3882 - accuracy: 0.8603 - val_loss: 0.4435 - val_accuracy: 0.8370
Epoch 3/5
970/970 [==============================] - 34s 35ms/step - loss: 0.3217 - accuracy: 0.8869 - val_loss: 0.4836 - val_accuracy: 0.8270
Epoch 4/5
970/970 [==============================] - 34s 35ms/step - loss: 0.2731 - accuracy: 0.9034 - val_loss: 0.5195 - val_accuracy: 0.8250
Epoch 5/5
970/970 [==============================] - 32s 33ms/step - loss: 0.2364 - accuracy: 0.9164 - val_loss: 0.5633 - val_accuracy: 0.8280


In [11]:
model.load_weights("./models_hdf5/LSTM-emb.hdf5")
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

32/32 [==============================] - 1s 16ms/step - loss: 0.4395 - accuracy: 0.8420
Validation Loss: 0.4395274221897125, Validation Accuracy: 0.8420000076293945


In [ ]:
model2 = Sequential()
model2.add(
    Embedding(
        input_dim=len(tokenizer.word_index) + 1,
        output_dim=300,
        input_length=X_train.shape[1],
    )
)
model2.add(SimpleRNN(64, activation="tanh", dropout=0.5))
model2.add(Dense(num_classes, activation="softmax"))
model2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

checkpoint = ModelCheckpoint(
    filepath="./models_hdf5/RNN-emb.hdf5",
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,
)

In [16]:
model2.fit(
    X_train,
    y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[checkpoint],
)

Epoch 1/5
970/970 [==============================] - 417s 430ms/step - loss: 0.5568 - accuracy: 0.7927 - val_loss: 0.4614 - val_accuracy: 0.8460
Epoch 2/5
970/970 [==============================] - 405s 417ms/step - loss: 0.4215 - accuracy: 0.8536 - val_loss: 0.4595 - val_accuracy: 0.8420
Epoch 3/5
970/970 [==============================] - 392s 404ms/step - loss: 0.3492 - accuracy: 0.8782 - val_loss: 0.5208 - val_accuracy: 0.8270
Epoch 4/5
970/970 [==============================] - 400s 413ms/step - loss: 0.2931 - accuracy: 0.9006 - val_loss: 0.5414 - val_accuracy: 0.8190
Epoch 5/5
970/970 [==============================] - 382s 394ms/step - loss: 0.2503 - accuracy: 0.9142 - val_loss: 0.5671 - val_accuracy: 0.8100


In [17]:
model2.load_weights("./models_hdf5/RNN-emb.hdf5")
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

32/32 [==============================] - 1s 15ms/step - loss: 0.4395 - accuracy: 0.8420
Validation Loss: 0.4395274221897125, Validation Accuracy: 0.8420000076293945


# TESTING

In [18]:
file_path = "test _no_label.csv"
df_test = pd.read_csv(file_path)
df_test

,ID,review_description
0,1,اهنئكم على خدمه العملاء في المحادثه المباشره م...
1,2,ممتاز جدا ولكن اتمنى ان تكون هناك بعض المسابقا...
2,3,كل محملته يقول تم ايقاف حطيت2 عشان تسوون الخطاء
3,4,شغل طيب
4,5,بعد ماجربت
...,...,...
995,996,يستهل
996,997,خدمة سيئة بكل المعايير
997,998,لؤي٠٣٣٢لؤ٣٤٣س
998,999,تطبيق غير صادق ف خصم الكوبونات


In [19]:
df_test["review_description"] = df_test["review_description"].apply(
    lambda text: text_cleaning(text)
)
X_test_seq = tokenizer.texts_to_sequences(df_test["review_description"])
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

In [20]:
y_pred_lstm = model.predict(X_test_pad)
y_pred_rnn = model2.predict(X_test_pad)

32/32 [==============================] - 2s 45ms/step


In [21]:
def return_prediction(y_pred):
    prediction = []
    for predict in y_pred:
        maxi = np.argmax(predict)
        if maxi == 2:
            prediction.append(1)
        elif maxi == 1:
            prediction.append(0)
        else:
            prediction.append(-1)
    return prediction


prediction_lstm = return_prediction(y_pred_lstm)
prediction_rnn = return_prediction(y_pred_rnn)

print(len(prediction_lstm), len(prediction_rnn))

1000 1000


In [22]:
results_df = pd.DataFrame({"ID": df_test["ID"], "rating": prediction_lstm})
results_df.to_csv("./results/test_results_LSTM_emb.csv", index=False)
results_df = pd.DataFrame({"ID": df_test["ID"], "rating": prediction_rnn})
results_df.to_csv("./results/test_results_rnn_emb.csv", index=False)

In [23]:
comparison_result = np.array(prediction_lstm) == np.array(prediction_rnn)
not_equal_idx = np.where(comparison_result != True)
print("The equal values are : ", np.sum(comparison_result))

The equal values are :  926


In [24]:
for idx in not_equal_idx[0]:
    text = df_test.iloc[idx, 1]
    rnn_prediction = prediction_rnn[idx]
    lstm_prediction = prediction_lstm[idx]
    print(f"Text: {text}, RNN: {rnn_prediction}, LSTM: {lstm_prediction}")

Text: عيش صلل يعن ما الا ثلث طعم صلل ليش صلل وجد طعم أرجو حل شكل, RNN: -1, LSTM: 0
Text: يا ليت وصل سلط عمن, RNN: -1, LSTM: 1
Text: بأس بس رات طلب ولم خار خلص يقل طعم غلق, RNN: -1, LSTM: 1
Text: انه حدد وقع فيم, RNN: -1, LSTM: 1
Text: طلب اخر وقت ومف تبع درة, RNN: -1, LSTM: 1
Text: بطل يجي عند رمز لام لما طلب ولا حدا برن شو شكل, RNN: -1, LSTM: 1
Text: سمع وصل قرة, RNN: -1, LSTM: 1
Text: فضل ليه افش حفظ نيا ركز رجء ضعه خطه, RNN: -1, LSTM: 1
Text: اخر بعض طعم وصل اكل برد, RNN: -1, LSTM: 1
Text: كثر زيد سعر وصل عرر ظعف محل, RNN: -1, LSTM: 1
Text: يري سهل ان اى حد قدر يبع محفظتى فلس ورى غير ما فتح حفظ, RNN: -1, LSTM: 1
Text: ضفة خصي دفع بطق ئتم لكل طعم حدد وقع طعم, RNN: -1, LSTM: 1
Text: رحب انا ارد شيء نقص طعم mcdonald s ليش ما بيج ماك تشك بس لحم لزم كمن دجج بيج ماك تشك رجء صلح شكل, RNN: -1, LSTM: 1
Text: ك ناى ؤءء ذ ءذ ءذذء ء ء ء ء د ء خزن زعزوع ء ء ء ىط ء ء ء ء ء, RNN: 1, LSTM: -1
Text: طلب ادر نهم رسل طلب بس معا ذلك طلب ديم تخر لو طلب انت ترح ولل سرع شكل فرق 45 دقق قلل ما اني طلب ثير ب